https://apps.bea.gov/iTable/iTable.cfm?reqid=147&step=51&isuri=1&table_list=13

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

qtrsrm = {'I': 1, 'II': 4, 'III': 7, 'IV': 10}

In [2]:
s = ['Quarter', 'Government', 'Utilities', 'Wholesale trade', 'Retail trade', 
     'Transportation and warehousing', 'Private goods-producing industries [1]',
     'Private services-producing industries [2]',
     'Finance, insurance, real estate, rental, and leasing',
     'Educational services, health care, and social assistance', 
     'Construction', 'Manufacturing', 'Information', 
     'Professional and business services']

d = {}
for dset in [1, 2, 3]:
    # Adjust top and bottom line 1, 2, 3 (downloaded data manually)
    df = pd.read_csv(f'raw/gdpbyindustry{dset}.csv', skiprows=4).T

    columns = df.iloc[1].str.strip()
    if dset == 3:
        columns[0] = 'Quarter'

    df.columns = columns

    si = [i for i in s if i in df.keys()]
    df = df.iloc[2:].loc[:, si]

    if dset == 3:
        dates = list(zip([dt[:4] for dt in df.index], df.Quarter.map(qtrsrm).values))
        date_index = pd.to_datetime([f'{year}-{month}-01' for year, month in dates])

        df = (df.reset_index().drop(['index', 'Quarter'], axis=1)
                .set_index(date_index).astype(float))

    else:
        dates = pd.to_datetime([f'{year}-01-01' for year in df.index])
        df = (df.reset_index().drop(['index'], axis=1)
                .set_index(dates).astype(float))    

    df.index.name = 'date'
    df['TTU'] = df[['Utilities', 'Wholesale trade', 'Retail trade', 
                    'Transportation and warehousing']].sum(axis=1)

    df['Oth_Serv'] = (df['Private services-producing industries [2]'] - df['TTU']
                      - df['Finance, insurance, real estate, rental, and leasing'])

    df['FIRE'] = df['Finance, insurance, real estate, rental, and leasing']

    df['GoodsTTU'] = df['Private goods-producing industries [1]'] + df['TTU']

    d[dset] = df

In [3]:
r = (d[1].append(d[2]).loc['1989':'2005'].resample('QS').pad().iloc[:-1]
    .append((d[1].append(d[2]).loc['2005'] - (d[3].loc['2005'].sum() / 3)).append(d[3])))

(r[['Government', 'Oth_Serv', 'FIRE', 'GoodsTTU']]
 .to_csv(data_dir / 'gdpva.csv', index_label='date', float_format='%g'))

(r[['Government', 'Oth_Serv', 'FIRE', 'GoodsTTU', 
    'Educational services, health care, and social assistance', 
    'Construction', 'Manufacturing', 'Retail trade', 'Information',
    'Professional and business services']]
 .to_csv(data_dir / 'gdpvafull.csv', index_label='date', float_format='%g'))

In [4]:
#list(pd.read_csv(f'raw/gdpbyindustry{dset}.csv', skiprows=4)['Unnamed: 1'])